In [1]:
from tkinter import *
from PIL import ImageTk,Image
from pymongo import MongoClient
import pprint
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt 
import time
import numpy as np
import datetime
from time import gmtime, strftime
from datetime import date
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  
NavigationToolbar2Tk) 

client = MongoClient('mongodb://localhost:27017/admin?readPreference=primary&appname=MongoDB%20Compass&ssl=false')
data=client['cs535']['product']
datac=client['cs535']['customer']

In [2]:
root=Tk()
root.title("Retail Shop Inventory Manager")
root.geometry("600x600")


''

In [3]:
def submit():
    mylist={"_id":prod_id.get(),"Description":des.get(),"Quantity":q.get(),"Price":price.get()}
    data.insert_one(mylist)
    des.delete(0,END)
    q.delete(0,END)
    price.delete(0,END)
def query():
    print_records=''
    st=inp.get()
    records = data.find_one({"_id": st})
    print(records)
    p = Label(root,text="Product ID")
    p.grid(row=9,column=0)

    d = Label(root,text="Description")
    d.grid(row=9,column=1)

    qq = Label(root,text="Quantity")
    qq.grid(row=9,column=2)

    pp = Label(root,text="Price")
    pp.grid(row=9,column=3,pady=10,padx=10,ipadx=100)

    self_e = Entry(root, width=10, fg='blue', font=('Arial',10,'bold'))       
    self_e.grid(row=10, column=0) 
    self_e.insert(END, records["_id"]) 

    e = Entry(root, width=30, fg='blue', font=('Arial',10,'bold'))       
    e.grid(row=10, column=1) 
    e.insert(END, records["Description"]) 

    ee = Entry(root, width=13, fg='blue', font=('Arial',10,'bold'))       
    ee.grid(row=10, column=2) 
    ee.insert(END, records["Quantity"]) 

    eee = Entry(root, width=13, fg='blue', font=('Arial',10,'bold'))       
    eee.grid(row=10, column=3) 
    eee.insert(END, records["Price"])
    
def invoice():
    mlist={"_id":inv_id.get(),"product_id":pro.get(),"Date":datetime.datetime.now(),"Quantity":qty.get(),"Unit Price":u_price.get(),"Total Cost":tot_price.get(),"CustomerID":cust_id.get(),"Country":con.get()}
    datac.insert_one(mlist)
    inv_id.delete(0,END)
    pro.delete(0,END)
    qty.delete(0,END)
    u_price.delete(0,END)
    tot_price.delete(0,END)
    cust_id.delete(0,END)
    con.delete(0,END)
    conf_label = Label(root,text="Generated Successfully")
    conf_label.grid(row=27,column =0)
    
def graph():
    figure = Figure(figsize=(6, 3.5), dpi=65)
    plot = figure.add_subplot(1, 1, 1)
    tt=data.find_one({ "_id": { "$eq": inp1.get() }},{"Description":1 ,"_id":0})
    label_title_l=tt['Description']
    pipeline = [
     {
         "$match": { "product_id": { "$eq": inp1.get() }
     }   
     }
    ,{
    "$group":{ 
        "_id":{
            "product_id":"$product_id",
            "pur_date":{
                '$dateToString':{
                    'format':"%m",
                    'date':"$Date"
                }
            }
        } ,   
        "total":{"$sum":"$Quantity"}	
    }
    }
    ,{
        "$project":{"_id.pur_date":1,"total":1}
    }]
    
    rm=list(client.cs535.customer.aggregate(pipeline))
    mon=[m['_id']['pur_date']for m in rm]
    tot=[m['total']for m in rm]
    mon_name=[]
    for i in range (0,len(mon)):
        datetime_object = datetime.datetime.strptime(mon[i], "%m")
        full_month_name = datetime_object.strftime("%B")
        mon_name.append(full_month_name)
    plot.plot(mon_name,tot)
    plot.set_ylabel('Sales')
    plot.set_xlabel('Months')
    plot.set_title(label_title_l)
    canvas = FigureCanvasTkAgg(figure, root)
    canvas.get_tk_widget().grid(row=30, column=2)

In [4]:
t_label = Label(root,text="Product Table")
t_label.grid(row=0,column=0,columnspan=2)

prod_id = Entry(root,width=30)
prod_id.grid(row=1,column=1,padx=30)

des = Entry(root,width=30)
des.grid(row=2,column=1)

q = Entry(root,width=30)
q.grid(row=3,column=1)

price = Entry(root,width=30)
price.grid(row=4,column=1)

prod_id_label = Label(root,text="Product ID")
prod_id_label.grid(row=1,column =0)

des_label = Label(root,text="Description")
des_label.grid(row=2,column =0)

q_label = Label(root,text="Quantity")
q_label.grid(row=3,column =0)

price_label = Label(root,text="Price")
price_label.grid(row=4,column =0)

submit_btn = Button (root,text="Add Record to Database",command=submit)
submit_btn.grid(row=5,column=0,columnspan=2,pady=10,padx=10,ipadx=100)

In [5]:
inp = Entry(root,width=30)
inp.grid(row=7,column=1,padx=30)

inp_label = Label(root,text="Enter Product ID")
inp_label.grid(row=7,column =0)

query_btn=Button (root,text ="Show Records",command = query)
query_btn.grid(row=8,column=0,columnspan=2,pady=10,padx=10,ipadx=137)

In [6]:
t1_label = Label(root,text="Invoice Generation")
t1_label.grid(row=17,column=0,columnspan=2)

inv_id = Entry(root,width=30)
inv_id.grid(row=18,column=1)

pro = Entry(root,width=30)
pro.grid(row=19,column=1)

qty = Entry(root,width=30)
qty.grid(row=20,column=1)

u_price = Entry(root,width=30)
u_price.grid(row=21,column=1)

tot_price = Entry(root,width=30)
tot_price.grid(row=22,column=1)

cust_id = Entry(root,width=30)
cust_id.grid(row=23,column=1)

con = Entry(root,width=30)
con.grid(row=24,column=1)

inv_id_label = Label(root,text="Invoice Number")
inv_id_label.grid(row=18,column =0)

pro_label = Label(root,text="Product Code")
pro_label.grid(row=19,column =0)

qty_label = Label(root,text="Quantity")
qty_label.grid(row=20,column =0)

u_price_label = Label(root,text="Unit Price")
u_price_label.grid(row=21,column =0)

tot_price_label = Label(root,text="Total Price")
tot_price_label.grid(row=22,column =0)

cust_id_label = Label(root,text="CustomerID")
cust_id_label.grid(row=23,column =0)

con_label = Label(root,text="Country")
con_label.grid(row=24,column =0)

inv_btn = Button (root,text="Generate Invoice",command=invoice)
inv_btn.grid(row=25,column=0,columnspan=2,pady=10,padx=10,ipadx=100)

inp1 = Entry(root,width=30)
inp1.grid(row=28,column=1,padx=30)

inp1_label = Label(root,text="Enter Product ID")
inp1_label.grid(row=28,column =0)

my_button=Button(root,text="See Trends",command=graph)
my_button.grid(row=29,column=0,columnspan=2,pady=10,padx=10,ipadx=130)



In [7]:
root.mainloop()

{'_id': '85123A', 'Description': 'WHITE HANGING HEART T-LIGHT HOLDER', 'Quantity': 504, 'Price': 2.55}
